In [47]:
import pandas as pd
from sklearn.cluster import KMeans,SpectralClustering,DBSCAN
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import calmap
from sklearn import metrics
from sklearn.preprocessing import scale
from sklearn import linear_model

In [8]:
from sklearn.model_selection import cross_val_score,train_test_split

In [4]:
def make_dataframe(f):
    df = pd.read_csv(f)
    print("imported file. converting datetime")
    if 'time' in df.columns.values:
        df['dt'] = pd.to_datetime(df['date'] + ' ' + df['time'], dayfirst=True)
    else:
        df['dt'] = pd.to_datetime(df['d'] + ' ' + df['t'],dayfirst=True)
    df['dt'] = df['dt'].dt.round('min')
    return df

In [3]:
f = "data/alliance_sub.csv"

In [2]:
df = make_dataframe(f)

NameError: name 'make_dataframe' is not defined

In [9]:
df.columns.values

array(['Unnamed: 0', 'd', 't', 'date', 'o3', 'no2', 'no', 'so2', 'temp',
       'humidity', 'pm1', 'pm25', 'pm10', 'bin0', 'bin1', 'bin2', 'bin3',
       'bin4', 'bin5', 'bin6', 'bin7', 'bin8', 'bin9', 'bin10', 'bin11',
       'bin12', 'bin13', 'bin14', 'bin15', 'd0', 'd1', 'd2', 'd3', 'd4',
       'd5', 'd6', 'd7', 'd8', 'd9', 'd10', 'd11', 'd12', 'd13', 'd14',
       'd15', 'dt'], dtype=object)

In [9]:
df = df.loc[df['no2']>0]

In [10]:
features = ['temp', 'humidity', 'd0', 'd1', 'd2', 'd3', 'd4','d5', 'd6', 'd7',\
            'd8', 'd9', 'd10', 'd11', 'd12', 'd13', 'd14', 'd15']

In [11]:
X = df[features].as_matrix()

In [59]:
X_sc = scale(X)

In [60]:
X

array([[   16.64     ,    85.37     ,  2444.13604  , ...,     0.       ,
            0.       ,     0.       ],
       [   16.64     ,    85.43     ,  2313.083169 , ...,     0.       ,
            0.       ,     0.       ],
       [   16.64     ,    85.09     ,  2424.47811  , ...,     0.       ,
            0.       ,     0.       ],
       ..., 
       [   15.96     ,    60.13     ,   576.6326315, ...,     0.       ,
            0.       ,     0.       ],
       [   15.95     ,    60.22     ,   727.3434329, ...,     0.       ,
            0.       ,     0.       ],
       [   15.96     ,    60.37     ,   583.185275 , ...,     0.       ,
            0.       ,     0.       ]])

In [61]:
X_sc

array([[-0.39551909,  1.58409614,  0.30076284, ..., -0.05691875,
        -0.04800277, -0.03600504],
       [-0.39551909,  1.58883005,  0.23640442, ..., -0.05691875,
        -0.04800277, -0.03600504],
       [-0.39551909,  1.56200456,  0.29110908, ..., -0.05691875,
        -0.04800277, -0.03600504],
       ..., 
       [-0.57951147, -0.40730169, -0.61634455, ..., -0.05691875,
        -0.04800277, -0.03600504],
       [-0.58221724, -0.40020083, -0.54233237, ..., -0.05691875,
        -0.04800277, -0.03600504],
       [-0.57951147, -0.38836605, -0.61312663, ..., -0.05691875,
        -0.04800277, -0.03600504]])

In [104]:
y_so2 = df['no2'].as_matrix()

In [105]:
X_train, X_test, y_train, y_test = train_test_split(X_sc, y_so2, test_size=0.20, random_state=42)

In [101]:
lr = linear_model.LinearRegression()
lr.fit(X_train,y_train)
y_ptrain = lr.predict(X_train)
y_pred = lr.predict(X_test)


In [63]:
lr.coef_

array([ 3.7639773 ,  3.74014012,  3.97152904,  4.46965496, -4.90733602,
        3.42388109,  0.71627932, -1.62858274,  0.59548715,  0.84522513,
       -0.08724129, -0.38627481, -0.31064626, -0.16265987,  0.06560969,
        0.20289816, -0.30835806,  0.1086909 ])

In [115]:
ridge = linear_model.Ridge(alpha = 100)
ridge.fit(X_train,y_train)
y_ptrain = ridge.predict(X_train)
y_pred = ridge.predict(X_test)

In [107]:
ridge.coef_

array([ 1.09241825, -1.41241394,  0.70995357, -0.44823468,  0.66549579,
       -0.39814528,  0.07020246, -0.27003374,  0.28893154,  0.30782827,
        0.07315956, -0.16477679, -0.07436463, -0.08663442,  0.02520171,
       -0.0167683 , -0.04876337,  0.01440012])

In [116]:
mse_tr = metrics.mean_squared_error(y_train,y_ptrain)

mse_te = metrics.mean_squared_error(y_test,y_pred)
r2 = metrics.r2_score(y_test, y_pred)
print mse_tr, mse_te, r2

28.0841006694 28.1839858825 0.170286266832


In [55]:
lr = linear_model.LinearRegression()
cv1 = cross_val_score(lr, X, y_so2, scoring = "r2", cv=10)
cv2 = cross_val_score(lr, X, y_so2, scoring = "neg_mean_squared_error", cv=10)
print zip(cv1,cv2)


[(-2.5819914297342952, -683.4708820565744), (0.12086042356925586, -104.25965307320099), (0.39090259357203982, -61.354333696517116), (0.060644583144931374, -98.795618537854494), (-0.022937406044141406, -101.7890162908417), (-0.30203663986632123, -112.6940783990178), (0.0021330337410632172, -106.75743788463599), (-0.39726289802023262, -115.30223336927685), (-0.14135985361925085, -80.858776136052143), (0.0377221598355858, -87.036551618958583)]


In [82]:
ridge = linear_model.Ridge(alpha = 1000)
cv1 = cross_val_score(ridge, X, y_so2, scoring = "r2", cv=10)
cv2 = cross_val_score(ridge, X, y_so2, scoring = "neg_mean_squared_error", cv=10)

print zip(cv1,cv2)


[(-1.786349172037041, -662.9116925299785), (0.37128958411597557, -185.15049399937064), (0.29668337075063544, -167.79712430543677), (0.066599976153138729, -122.38511729710835), (-0.07941521686484565, -110.07538464002837), (-0.14749781826001374, -113.70146926397869), (-0.58579452431525891, -97.756455466660526), (-0.78288482622368805, -120.98197102186992), (-0.26168724144455191, -80.78985453883331), (-0.21842873978443311, -76.620309944473661)]


In [71]:
y_no = df['no2'].as_matrix()

In [73]:
cv1 = cross_val_score(lr, X, y_no, scoring = "r2", cv=10)
cv2 = cross_val_score(lr, X, y_no, scoring = "neg_mean_squared_error", cv=10)

print zip(cv1,cv2)


[(-0.14003139523854835, -35.058931924355925), (0.14181860232014354, -33.103380863368038), (0.24761756202115437, -24.870476853084764), (0.22116967594936787, -23.709674031242479), (0.11351612662694832, -27.892138012861356), (-0.049029842323369488, -23.970306755263564), (-0.025736560049259838, -23.609575227507992), (-0.022047502439754174, -30.238700571457166), (-0.10371830086142309, -47.379996274825437), (0.076670062481685286, -18.072286619299522)]


In [61]:
from sklearn.neural_network import MLPRegressor
mlp = MLPRegressor(hidden_layer_sizes=(100,50,))

In [ ]:

cv2 = cross_val_score(mlp, X, y_no, scoring = "neg_mean_squared_error", cv=10)

print cv2


NameError: name 'pd' is not defined